In [2]:
import numpy as np
import plotly.graph_objects as go

from multiprocess import Pool
from functools import partial

import games

import cpb_gaussian

import os
import gzip
import pickle as pkl
import cpb_side_gaussian
import cpb_side
import random_algo

def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, horizon) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result[i] = pkl.load(f)
            except EOFError:
                break
    return result

name = 'LE'
task = 'easy'
context_type = 'linear'

n_folds = 96
horizon = 100000
d = 2


colors = [   [250,0,0], [0,250,0]  ] 
labels = [   'CBPside005',  'RandCBPside005_1_5_07' ]
labels_latex = [ 'CBPside',  'RandCBPside'  ] 

fig = go.Figure( )

for color, label, label_latex in zip( colors, labels, labels_latex):
    r,g,b = color
    direct = './contextual_results/{}'.format(name)
    # if label_latex == 'RandCBPside':
    #     path = os.path.join(direct, 'benchmark_{}_{}_{}_{}_{}.pkl.gz'.format(task, context_type, horizon, n_folds, label) )
    # else:
    path = os.path.join(direct, '{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )

    result = loadall_results(path, horizon, n_folds)

    worst_case = np.argmax( result[:,-1] )
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    #std =  result[worst_case]  #np.std(result,0) 
    upper_regret = result[worst_case] #regret + std

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label_latex,legendgrouptitle_text='Algorithm',legendgroup='Algorithm'  )) # 
    
    if 'random' not in label_latex:

        fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb({},{},{})'.format(r,g,b), width = 2, dash=  'dot' ), mode='lines',
                legendgroup='Worst Case', name=label_latex, showlegend = True, legendgrouptitle_text='Worst Case',    ) ) 

    # else:
    #     if 'CBPside' in label_latex:
    #         fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb(192,192,192,0.7)', width = 0.5, dash=  'solid' ), mode='lines',
    #             legendgroup='Worst Case', name='CBPside', showlegend = True    ) ) 
        # else:
        #     fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb(192,192,192,0.5)', width = 0.5, dash=  'solid' ), mode='lines', showlegend=False  ) ) 


fig.update_layout(legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.05), autosize=False,
                 xaxis_title="Sequence",
                 yaxis_title="Regret",
                 margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                  font=dict(size=13,) )

fig.update_yaxes(range=[0, 800] )
fig.show(legend=True)

# fig.write_image("./{}_{}_{}.pdf".format(task,name, context_type)  )

# fig.update_yaxes(type="log")
# fig.show(legend=True)

# fig.write_image("./{}_{}_{}.pdf".format(task,name, context_type)  )

#fig.update_xaxes(type="log")
# fig.write_image("./easy_AT_context_log.pdf")


FileNotFoundError: [Errno 2] No such file or directory: './contextual_results/LE/easy_linear_100000_96_RandCBPside005_1_5_07.pkl.gz'

In [ ]:
name = 'AT'
task = 'easy'
context_type = 'quintic'


n_folds = 96
horizon = 100000

colors = [  [0,0,0],  [250,0,0], [0,250,0]  ] 
labels = [  'random', 'CBPside005',  'RandCBPside005' ]
labels_latex = [  'random', 'CBPside',  'RandCBPside'  ] 

final_regrets = {}

for label, label_latex in zip(labels, labels_latex):

    direct = './contextual_results/{}'.format(name)
    path = os.path.join(direct, '{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )
    result = loadall_results(path, horizon, n_folds)
    final_regrets[label_latex] =  result[:,-1] 

final = np.array( [ (1, i) if final_regrets['RandCBPside'][i] <= final_regrets['CBPside'][i] else (0, i) for i in range(n_folds) ] )

print( sum(final[:,0]) )



In [ ]:
import numpy as np
import plotly.graph_objects as go

from multiprocess import Pool
from functools import partial

import games

import cpb_gaussian

import os
import gzip
import pickle as pkl
import cpb_gaussian


def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, horizon) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result[i] = pkl.load(f)
            except EOFError:
                break
    return result

name = 'LE'
task = 'difficult'
context_type = 'quintic'
epsilon = 10e-7 #0.1
game = games.apple_tasting(False)

n_folds = 96
horizon = 100000
d = 2


algos = [
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1, 5, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/8, 5, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/16, 5, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/32, 5, False, epsilon), 

          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1, 10, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/8, 10, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/16, 10, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/32, 10, False, epsilon), 

          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1, 20, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/8, 20, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/16, 20, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/32, 20, False, epsilon), 
          
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1, 100, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/8, 100, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/16, 100, False, epsilon),
          cpb_side_gaussian.RandCPB_side(  game,d, horizon, 1.01, False, 1/32, 100, False, epsilon), ]



colors = [  [250,0,0],[250,0,0], [250,0,0], [250,0,0],
            [0,250,0], [0,250,0], [0,250,0], [0,250,0],
            [0,0,250], [0,0,250], [0,0,250],[0,0,250],
            [255,128,0], [255,128,0], [255,128,0], [255,128,0]  ]

labels_latex = [   'Gaussian_1_5_10e7',  'Gaussian_18_5_10e7', 'Gaussian_116_5_10e7','Gaussian_132_5_10e7', 
             'Gaussian_1_10_10e7',  'Gaussian_18_10_10e7', 'Gaussian_116_10_10e7','Gaussian_132_10_10e7',
             'Gaussian_1_20_10e7',  'Gaussian_18_20_10e7', 'Gaussian_116_20_10e7','Gaussian_132_20_10e7', 
             'Gaussian_1_100_10e7',  'Gaussian_18_100_10e7', 'Gaussian_116_100_10e7','Gaussian_132_100_10e7'  ]
             
            #  'Gaussian_1_5_01',  'Gaussian_18_5_01', 'Gaussian_116_5_01','Gaussian_132_5_01', 
            #  'Gaussian_1_10_01',  'Gaussian_18_10_01', 'Gaussian_116_10_01','Gaussian_132_10_01',
            #  'Gaussian_1_20_01',  'Gaussian_18_20_01', 'Gaussian_116_20_01','Gaussian_132_20_01', 
            #  'Gaussian_1_100_01',  'Gaussian_18_100_01', 'Gaussian_116_100_01','Gaussian_132_100_01',  ] 


labels = [ 'RandCBPside005_1_5_07', 'RandCBPside005_18_5_07', 'RandCBPside005_116_5_07', 'RandCBPside005_132_5_07',
                 'RandCBPside005_1_10_07', 'RandCBPside005_18_10_07', 'RandCBPside005_116_10_07', 'RandCBPside005_132_10_07',
                  'RandCBPside005_1_20_07', 'RandCBPside005_18_20_07', 'RandCBPside005_116_20_07', 'RandCBPside005_132_20_07',
                   'RandCBPside005_1_100_07', 'RandCBPside005_18_100_07', 'RandCBPside005_116_100_07', 'RandCBPside005_132_100_07'  ]

            # 'RandCBPside005_1_5_01', 'RandCBPside005_18_5_01', 'RandCBPside005_116_5_01', 'RandCBPside005_132_5_01',
            #          'RandCBPside005_1_10_01', 'RandCBPside005_18_10_01', 'RandCBPside005_116_10_01', 'RandCBPside005_132_10_01',
            #           'RandCBPside005_1_20_01', 'RandCBPside005_18_20_01', 'RandCBPside005_116_20_01', 'RandCBPside005_132_20_01', 
            #           'RandCBPside005_1_100_01' 'RandCBPside005_18_100_01' 'RandCBPside005_116_100_01' 'RandCBPside005_132_100_01'

fig = go.Figure( )


for alg, color, label, label_latex in zip( algos, colors, labels, labels_latex):
    r,g,b = color
    direct = './contextual_results/{}'.format(name)
    path = os.path.join(direct, 'benchmark_{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )

    result = loadall_results(path, horizon, n_folds)

    worst_case = np.argmax( result[:,-1] )
    # print(worst_case)
    
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    #std =  result[worst_case]  #np.std(result,0) 
    upper_regret = result[worst_case] #regret + std
    
    if 'Gaussian_132' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/32'  ) ) 

    elif 'Gaussian_116' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdashdot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/16'   ) )  

    elif 'Gaussian_18' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dash' ), mode='lines',
                legendgroup='M = {}'.format( label.split('_')[2] ),  name='σ = 1/8'   ) ) 

    elif 'Gaussian_1' in label_latex:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ),legendgrouptitle_text='M = {}'.format( label.split('_')[2] ), name='σ = 1'  ) ) 



for alg, color, label, label_latex in zip( algos, colors, labels, labels_latex):
    r,g,b = color
    direct = './contextual_results/{}'.format(name)
    path = os.path.join(direct, 'benchmark_{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )

    result = loadall_results(path, horizon, n_folds)

    worst_case = np.argmax( result[:,-1] )
    
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    #std =  result[worst_case]  #np.std(result,0) 
    upper_regret = result[worst_case] #regret + std

    if 'Gaussian_1_5' in label_latex:
        # fig.add_trace( go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba(0,0,0,0.1)'.format(r,g,b), 
        #                   line=dict(color='rgba(0,0,0,0)'), legendgrouptitle_text='Worst Case',  hoveron='points', hoverinfo="skip",  showlegend=True, name = 'of best algo' ) )
        fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb(0,0,0)', width = 0.5, dash=  'solid' ), mode='lines',
                legendgroup='Worst Case', name='of best algo', showlegend = True,legendgrouptitle_text='Worst Case',    ) ) 

    else:
        if 'Gaussian_132_100' in label_latex:
            fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb(192,192,192,0.5)', width = 0.5, dash=  'solid' ), mode='lines',
                legendgroup='Worst Case', name='of other algos', showlegend = True    ) ) 
        else:
            fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb(192,192,192,0.5)', width = 0.5, dash=  'solid' ), mode='lines', showlegend=False  ) ) 



fig.update_yaxes(range=[0, 20000] )

fig.update_layout(  legend= dict(yanchor="top",y=0.97,xanchor="left",x=0.03), #x = 0.25, y=0.15 for center, x= 0.97, y=0.03
                        legend_orientation="h",
                        autosize=False,
                        xaxis_title="Sequence",
                        yaxis_title="Mean Cum. Regret + Worst Cases",
                        margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),  )
fig.show()

fig.write_image("./benchmark_{}_{}_{}_{}.pdf".format(task, name, context_type, epsilon)  )

# fig.update_yaxes(type="log")
# fig.show(legend=True)

# fig.write_image("./benchmark_{}_{}_{}_{}.pdf".format(task, name, context_type, epsilon)  )
